In [8]:

import pandas as pd
from sklearn.preprocessing import StandardScaler
df = pd.read_pickle('dataset/clean_dataset.pkl')


In [9]:
# cols = ['FIT101', 'LIT101', 'MV101', 'P101', 'P102', 'AIT201',
#        'AIT202', 'AIT203', 'FIT201', 'MV201', 'P201', 'P202', 'P203', 'P204',
#        'P205', 'P206', 'DPIT301', 'FIT301', 'LIT301', 'MV301', 'MV302',
#        'MV303', 'MV304', 'P301', 'P302', 'AIT401', 'AIT402', 'FIT401',
#        'LIT401', 'P401', 'P402', 'P403', 'P404', 'UV401', 'AIT501', 'AIT502',
#        'AIT503', 'AIT504', 'FIT501', 'FIT502', 'FIT503', 'FIT504', 'P501',
#        'P502', 'PIT501', 'PIT502', 'PIT503', 'FIT601', 'P601', 'P602', 'P603']

# cols = [i for i in cols if i not in to_remove]
# feature = ['PIT503']

def get_train_test_dataset():
    split_date = '2015-12-28 10:29:13'
    data_train = df.loc[df['Timestamp'] <= split_date].copy()
    data_test = df.loc[df['Timestamp'] > split_date].copy()

    return data_train, data_test

In [10]:
training, test = get_train_test_dataset()

cols_numeric = list(df.columns)

cols_numeric.remove('index')
cols_numeric.remove('attack')

# cols_numeric.remove('Normal/Attack')
data = training[cols_numeric]
test = test[cols_numeric]

#Transform to numeric data to be able to do the SC
data['Timestamp'] = pd.to_numeric(data['Timestamp'])
data['Normal/Attack'].replace(['Normal', 'Attack'],
                        [0, 1], inplace=True)

test['Timestamp'] = pd.to_numeric(test['Timestamp'])
test['Normal/Attack'].replace(['Normal', 'Attack'],
                        [0, 1], inplace=True)

train_set = data
test_set = test

# We only fit on the trainset to avoid data leakage from the testset
sc = StandardScaler().fit(data)

train_set[0:len(train_set)-1] = sc.transform(train_set[0:len(train_set)-1])
test_set[0:len(test_set)-1] = sc.transform(test_set[0:len(test_set)-1])


/tmp/ipykernel_26029/3892842226.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Timestamp'] = pd.to_numeric(data['Timestamp'])
/tmp/ipykernel_26029/3892842226.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Normal/Attack'].replace(['Normal', 'Attack'],
/tmp/ipykernel_26029/3892842226.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.htm

In [11]:
import numpy as np

def create_dataset(X, y, time_steps=1):
    Xs, ys = [], []
    for i in range(len(X) - time_steps):
        v = X.iloc[i:(i + time_steps)].values
        Xs.append(v)
        ys.append(y.iloc[i + time_steps])

    return np.array(Xs), np.array(ys)

In [13]:

cols = ['FIT101', 'LIT101', 'MV101', 'P101', 'P102', 'AIT201',
       'AIT202', 'AIT203', 'FIT201', 'MV201', 'P201', 'P202', 'P203', 'P204',
       'P205', 'P206', 'DPIT301', 'FIT301', 'LIT301', 'MV301', 'MV302',
       'MV303', 'MV304', 'P301', 'P302', 'AIT401', 'AIT402', 'FIT401',
       'LIT401', 'P401', 'P402', 'P403', 'P404', 'UV401', 'AIT501', 'AIT502',
       'AIT503', 'AIT504', 'FIT501', 'FIT502', 'FIT503', 'FIT504', 'P501',
       'P502', 'PIT501', 'PIT502', 'PIT503', 'FIT601', 'P601', 'P602', 'P603']

to_remove = ['P202','P401','P404','P502','P601','P603']
cols = [i for i in cols if i not in to_remove]
feature = ['PIT503']
cols.remove(feature)

# size of sequence
TIME_STEPS = 300

# reshape to [samples, time_steps, n_features]
X_train, y_train = create_dataset(training[cols], training[feature], TIME_STEPS)
X_test, y_test = create_dataset(test[cols], test[feature], TIME_STEPS)
